# Examples of how to use widget:

Aditional dependencies for extra features

In [1]:
%pip install distinctipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# handle launching on colab
if "google.colab" in str(get_ipython()):
    # running on CoLab
    %pip install -U "ipylabel>=0.1.7"

    from google.colab import output

    output.enable_custom_widget_manager()

In [3]:
from __future__ import annotations

import ipylabel

## Extend widget

### Basic example

In [4]:
from traitlets import observe
from ipywidgets import Output


output = Output()


class MyExtendedWidget(ipylabel.TextWidget):
    def __init__(self, name: str, **kwargs):
        super().__init__(**kwargs)
        self.name = name

    @observe("finished", "result")
    @output.capture()
    def _finished_and_res_changed(self, change):
        if change["name"] == "finished" and change["new"]:
            answer = None
        else:
            answer = change["owner"].result
        print(answer)


my_extended_widget = MyExtendedWidget(
    "first widget",
    text="Мама мыла раму",
    labels=["protein", "DNA", "RNA"],
    colors=["#ff0000", "#00ff00", "#0000ff"],
)
display(my_extended_widget, output)

MyExtendedWidget(colors=['#ff0000', '#00ff00', '#0000ff'], labels=['protein', 'DNA', 'RNA'], text='Мама мыла р…

Output()

### More advanced example with more features

In [5]:
from traitlets import observe, default
from ipywidgets import Output
from distinctipy import distinctipy

output = Output()


class MyAdvancedExtendedWidget(ipylabel.TextWidget):
    def __init__(self, name: str, **kwargs):
        super().__init__(**kwargs)
        self.name = name

    @default("colors")
    def get_colors_default(self):
        # conversion taken from https://github.com/matplotlib/matplotlib/blob/v3.5.3/lib/matplotlib/colors.py#L389-L408
        return [
            "#" + "".join(format(int(round(val * 255)), "02x") for val in c)
            for c in distinctipy.get_colors(len(self.labels), rng=42)
        ]

    @observe("finished", "result")
    @output.capture()
    def _finished_and_res_changed(self, change):
        if change["name"] == "finished" and change["new"]:
            answer = None
        else:
            answer = change["owner"].result
        print(answer)


my_advanced_extended_widget = MyAdvancedExtendedWidget(
    "first widget",
    text="Мама мыла раму",
    labels=["protein", "DNA", "RNA", "RNA2", "RNA3"],
)
display(my_advanced_extended_widget, output)

MyAdvancedExtendedWidget(colors=['#00ff00', '#ff00ff', '#0080ff', '#ff8000', '#80bf80'], labels=['protein', 'D…

Output()

## Wrap widget

## Basic example

In [6]:
from traitlets import observe
from ipywidgets import Output

output = Output()


class MyWrappedWidget:
    def __init__(
        self,
        name: str,
        text: str,
        labels: list[str],
        colors: list[str],
    ):
        self.name = name
        self.widget = ipylabel.TextWidget(text=text, labels=labels, colors=colors)
        self.widget.observe(self._finished_and_res_changed, ["finished", "result"])

    @output.capture()
    def _finished_and_res_changed(self, change):
        if change["name"] == "finished" and change["new"]:
            answer = None
        else:
            answer = change["owner"].result
        print(answer)


my_wrapped_widget = MyWrappedWidget(
    "first widget",
    text="Мама мыла раму",
    labels=["protein", "DNA", "RNA"],
    colors=["#ff0000", "#00ff00", "#0000ff"],
)
display(my_wrapped_widget.widget, output)

TextWidget(colors=['#ff0000', '#00ff00', '#0000ff'], labels=['protein', 'DNA', 'RNA'], text='Мама мыла раму')

Output()

## Advanced example

In [7]:
from traitlets import observe
from ipywidgets import Output

output = Output()


class MyAdvancedWrappedWidget:
    def __init__(
        self,
        name: str,
        text: str,
        labels: list[str],
    ):
        self.name = name
        self.widget = ipylabel.TextWidget(
            text=text,
            labels=labels,
            colors=[
                "#" + "".join(format(int(round(val * 255)), "02x") for val in c)
                for c in distinctipy.get_colors(len(labels), rng=42)
            ],
        )
        self.widget.observe(self._finished_and_res_changed, ["finished", "result"])

    @output.capture()
    def _finished_and_res_changed(self, change):
        if change["name"] == "finished" and change["new"]:
            answer = None
        else:
            answer = change["owner"].result
        print(answer)

    def _repr_pretty_(self, pp, cycle):
        return display(self.widget)


my_advanced_wrapped_widget = MyAdvancedWrappedWidget(
    "first widget",
    text="Мама мыла раму",
    labels=["protein", "DNA", "RNA"],
)
display(my_advanced_wrapped_widget, output)

TextWidget(colors=['#00ff00', '#ff00ff', '#0080ff'], labels=['protein', 'DNA', 'RNA'], text='Мама мыла раму')

Output()